## Projet BiLSTM

Liste des imports

In [2]:
!pip3 install convokit

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of convokit to determine which version is compatible with other requirements. This could take a while.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached spacy-3.8.2.tar.gz (1.3 MB)
  Installing build dependencies ... error
  error: subprocess-exited-with-error
  
  × pip subprocess to install build dependencies did not run successfully.
  │ exit code: 1
  ╰─> [92 lines of output]
      Ignoring numpy: markers 'python_version >= "3.9"' don't match yo

In [1]:
import os
import re
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense
from sklearn.model_selection import train_test_split


2025-04-07 13:59:27.705593: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-07 13:59:28.112257: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-07 13:59:28.113768: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-04-07 13:59:29.224084: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [1]:
## Download the Cornell Movie Dialogs Corpus
from convokit import Corpus, download
corpus = Corpus(filename=download("movie_lines.txt"))

ModuleNotFoundError: No module named 'convokit'

In [2]:
import tensorflow as tf
print("GPU disponible :", tf.config.list_physical_devices('GPU'))


GPU disponible : []


In [ ]:
# Chargement des lignes
with open("movie_lines.txt", encoding='iso-8859-1') as file:
    lines = file.read().split("\n")

# Dictionnaire id -> ligne
id2line = {}
for line in lines:
    parts = line.split(" +++$+++ ")
    if len(parts) == 5:
        id2line[parts[0]] = parts[4]

# Chargement des conversations
with open("movie_conversations.txt", encoding='iso-8859-1') as file:
    conversations = file.read().split("\n")

# Création des paires question-réponse
questions = []
answers = []

for conv in conversations:
    parts = conv.split(" +++$+++ ")
    if len(parts) == 4:
        utterance_ids = eval(parts[3])
        for i in range(len(utterance_ids) - 1):
            if utterance_ids[i] in id2line and utterance_ids[i+1] in id2line:
                questions.append(id2line[utterance_ids[i]])
                answers.append(id2line[utterance_ids[i+1]])


In [ ]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r"[^a-zA-Z0-9?.!,¿]+", " ", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

questions = [clean_text(q) for q in questions]
answers = [clean_text(a) for a in answers]


In [ ]:
vocab_size = 10000
max_len = 40

tokenizer = Tokenizer(num_words=vocab_size, oov_token="<OOV>")
tokenizer.fit_on_texts(questions + answers)

question_seq = tokenizer.texts_to_sequences(questions)
answer_seq = tokenizer.texts_to_sequences(answers)

question_pad = pad_sequences(question_seq, maxlen=max_len, padding='post')
answer_pad = pad_sequences(answer_seq, maxlen=max_len, padding='post')


In [ ]:
model = Sequential([
    Embedding(vocab_size, 256, input_length=max_len),
    Bidirectional(LSTM(256, return_sequences=True)),
    Bidirectional(LSTM(256)),
    Dense(vocab_size, activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()


In [ ]:
X_train, X_val, y_train, y_val = train_test_split(question_pad, answer_pad, test_size=0.2)

# On prédit le premier mot de la réponse comme approximation simple
model.fit(X_train, np.expand_dims(y_train[:, 0], -1), 
          validation_data=(X_val, np.expand_dims(y_val[:, 0], -1)), 
          batch_size=128, epochs=10)


In [ ]:
def generate_reply(input_text):
    input_text = clean_text(input_text)
    seq = tokenizer.texts_to_sequences([input_text])
    padded = pad_sequences(seq, maxlen=max_len, padding='post')
    prediction = model.predict(padded)
    word_index = np.argmax(prediction[0])
    
    for word, index in tokenizer.word_index.items():
        if index == word_index:
            return word
    return "..."

# Exemple :
generate_reply("Are you ready for the adventure?")
